# Section 31: Support Vector Machines

## Learning Objectives

- To revisit Linear (Algebra) Equations and revisit the relationship between $y=mx+b$ and $y= w^TX+B$.
- To understand how support vector machine attempts to separate groups.
- Discuss the advantages / disadvantages of SVMs

- To understand the math notation of SVMs

- Apply SVMs in the SVMs In scikit Learn Lab


## Resources

- BLOG POSTS/ARTICLES
    - [Towards Data Science - SVM Simply Explained](https://towardsdatascience.com/support-vector-machine-simply-explained-fee28eba5496)
- STUDY GROUP RECORDINGS:
    - [Support Vector Machines - Victor](https://www.youtube.com/watch?v=_QmnoubpU3Q&list=PLVoXE6pv5LIg4WOllQ4rNPi9BtvtVMb78&index=5)
    - [The Kernel Trick - Victor](https://www.youtube.com/watch?v=mnN74NI4Gqk&list=PLVoXE6pv5LIg4WOllQ4rNPi9BtvtVMb78&index=6)

In [ ]:
!pip install -U fsds_100719
from fsds_100719.imports import *

# Back to Our Iowa Prisoners

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from fsds_100719.imports import *

### previous functions

In [ ]:
# fs.quick

In [ ]:
## a timer to record how long a process takes
class Timer():
    ## def init
    def __init__(self,format_="%m/%d/%y - %I:%M %p"):
        import tzlocal
        self.tz = tzlocal.get_localzone()
        self.fmt = format_
        
        self.created_at = self.get_time()# get time
    
    ## def get time method
    def get_time(self):
        import datetime as dt
        return dt.datetime.now(self.tz)

    ## def start
    def start(self):
        time = self.get_time()
        self.start = time
        print(f"[i] Timer started at {self.start.strftime(self.fmt)}")
        
        

    ## def stop
    def stop(self):
        time = self.get_time()
        self.end = time
        print(f"[i] Timer ended at {self.end.strftime(self.fmt)}")
        print(f"- Total time = {self.end-self.start}")
timer = Timer()
print(timer.created_at)
timer.start()
timer.stop()

In [ ]:
def plot_importance(tree, top_n=20,figsize=(10,10)):
    df_importance = pd.Series(tree.feature_importances_,index=X_train.columns)
    df_importance.sort_values(ascending=True).tail(top_n).plot(
        kind='barh',figsize=figsize)
    return df_importance

## Write a fucntion to evalute the model
import sklearn.metrics as metrics

def evaluate_model(y_true, y_pred,X_true,clf):
    
    ## Classification Report / Scores 
    print(metrics.classification_report(y_true,y_pred))

    fig, ax = plt.subplots(figsize=(10,4),ncols=2)
    metrics.plot_confusion_matrix(clf,X_true,y_true,cmap="Greens",
                                  normalize='true',ax=ax[0])
    ax[0].set(title='Confusion Matrix')
    y_score = clf.predict_proba(X_true)[:,1]

    fpr,tpr,thresh = metrics.roc_curve(y_true,y_score)
    # print(f"ROC-area-under-the-curve= {}")
    roc_auc = round(metrics.auc(fpr,tpr),3)
    ax[1].plot(fpr,tpr,color='darkorange',label=f'ROC Curve (AUC={roc_auc})')
    ax[1].plot([0,1],[0,1],ls=':')
    ax[1].legend()
    ax[1].grid()
    ax[1].set(ylabel='True Positive Rate',xlabel='False Positive Rate',
          title='Receiver operating characteristic (ROC) Curve')
    plt.tight_layout()
    plt.show()
    try: 
        df_important = plot_importance(clf)
    except:
        df_important = None
    
#     return df_important
## visualize the decision tree
def visualize_tree(tree,feature_names=None,class_names=['0','1'],
                   kws={},save_filename=None,format_='png',save_and_show=False):
    """Visualizes a sklearn tree using sklearn.tree.export_graphviz"""
    from sklearn.tree import export_graphviz
    from IPython.display import SVG
    import graphviz #import Source
    from IPython.display import display
    
    if feature_names is None:
        feature_names=X_train.columns

    tree_viz_kws =  dict(out_file=None,rounded=True, rotate=False, filled = True)
    tree_viz_kws.update(kws)

    # tree.export_graphviz(dt) #if you wish to save the output to a dot file instead
    tree_data=export_graphviz(tree,feature_names=feature_names, 
                                   class_names=class_names,**tree_viz_kws)
    graph = graphviz.Source(tree_data,format=format_)#'png')
    
    if save_filename is not None:
        graph.render(save_filename)
        if save_and_show:
            display(graph)
        else:
            print(f'[i] Tree saved as {save_filename}.{format_}')
    else:
        display(graph)

#     display(SVG(graph.pipe(format=format_)))#'svg')))

# Obtain

In [ ]:
df = fs.datasets.load_iowa_prisoners(vers='clean',read_csv_kwds={'index_col':0})
df.head()

In [ ]:
## Drop unwanted cols
df= df.drop(columns=['yr_released','report_year'])
df.info()

In [ ]:
## Check null values
import missingno
missingno.matrix(df)
df.isna().sum()

In [ ]:
df['race_ethnicity'].value_counts(dropna=False)

In [ ]:
# Defining Dictionary Map for race_ethnicity categories
race_ethnicity_map = {'White - Non-Hispanic':'White',
                        'Black - Non-Hispanic': 'Black',
                        'White - Hispanic' : 'Hispanic',
                        'American Indian or Alaska Native - Non-Hispanic' : 'American Native',
                        'Asian or Pacific Islander - Non-Hispanic' : 'Asian or Pacific Islander',
                        'Black - Hispanic' : 'Black',
                        'American Indian or Alaska Native - Hispanic':'American Native',
                        'White -' : 'White',
                        'Asian or Pacific Islander - Hispanic' : 'Asian or Pacific Islander',
                        'N/A -' : np.nan,
                        'Black -':'Black'}

df['race_ethnicity'] = df['race_ethnicity'].map(race_ethnicity_map)
df['race_ethnicity'].value_counts(dropna=False)

In [ ]:
df['crime_class'].value_counts()

In [ ]:
# Remapping
crime_class_map = {'Other Felony (Old Code)': np.nan ,#or other felony
                  'Other Misdemeanor':np.nan,
                   'Felony - Mandatory Minimum':np.nan, # if minimum then lowest sentence ==  D Felony
                   'Special Sentence 2005': 'Sex Offender',
                   'Other Felony' : np.nan ,
                   'Sexual Predator Community Supervision' : 'Sex Offender',
                   'D Felony': 'D Felony',
                   'C Felony' :'C Felony',
                   'B Felony' : 'B Felony',
                   'A Felony' : 'A Felony',
                   'Aggravated Misdemeanor':'Aggravated Misdemeanor',
                   'Felony - Enhancement to Original Penalty':'Felony - Enhanced',
                   'Felony - Enhanced':'Felony - Enhanced' ,
                   'Serious Misdemeanor':'Serious Misdemeanor',
                   'Simple Misdemeanor':'Simple Misdemeanor'}

df['crime_class'] = df['crime_class'].map(crime_class_map)
df['crime_class'].value_counts(dropna=False)

In [ ]:
df['age_released'].value_counts(dropna=False)

In [ ]:
# Encoding age groups as ordinal
age_ranges = ('Under 25','25-34', '35-44','45-54','55 and Older')
age_codes = (0,1,2,3,4) 
# Zipping into Dictionary to Map onto Column
age_map = dict(zip(age_ranges,age_codes))
age_map

In [ ]:
df['age_enc'] = df['age_released'].map(age_map)
df['age_enc'].value_counts()

In [ ]:
# Mapping age_map onto 'age_released'
# Encoding age groups as ordinal
age_ranges = ('Under 25','25-34', '35-44','45-54','55 and Older')
age_numbers = (20,30,40,50,70) 
age_num_map = dict(zip(age_ranges,age_numbers))
age_num_map

In [ ]:
df['age_number'] = df['age_released'].map(age_num_map)
df['age_number'].value_counts()

In [ ]:
## Drop Nulls 
print(df.isna().sum().divide(len(df))*100)
drop_cols  = [col for col in df.drop(columns=['super_dist','release_type']).columns]
drop_cols

In [ ]:
df.dropna(subset=drop_cols, inplace=True)
df.isna().sum()

In [ ]:
df[['super_dist','release_type']] = df[['super_dist','release_type']].fillna('Missing')
df.isna().sum()

In [ ]:
drop_cols = ['age_released']
df = df.drop(columns=drop_cols)

In [ ]:
# one_hot_cols = ['race_ethnicity','crime_class
df = pd.get_dummies(df)#,dummy_na=True)
df.head()

In [ ]:
df.drop('recidivist_No',axis=1,inplace=True)

In [ ]:
y = df.pop('recidivist_Yes')
df

In [ ]:
from sklearn.model_selection import train_test_split

## Train test split
X_train, X_test, y_train,y_test  = train_test_split(df,y,test_size=.3)
y_train.value_counts(normalize=True)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)
pd.Series(y_train).value_counts()

In [ ]:
# mpl.rcParams['figure.figsize'] =(20,20)
# pd.plotting.scatter_matrix(df);

In [ ]:
# px.scatter_matrix(df, color='recidivist')

In [ ]:
# pd.plotting.scatter_matrix(df,diagonal='kde');

In [ ]:
# px.scatter_matrix(df,color='recidivist',
#                  dimensions=['race_ethnicity','super_dist','age_released',
#                             'target_pop','crime_class','crime_class'])

## RandomForests Revisited

In [ ]:
from xgboost import XGBRFClassifier,XGBClassifier
## Fit and Evaluate

xgb_rf = XGBRFClassifier()
xgb_rf.fit(X_train, y_train)

print(xgb_rf.score(X_train,y_train))
print(xgb_rf.score(X_test,y_test))

y_hat_test = xgb_rf.predict(X_test)

evaluate_model(y_test,y_hat_test,X_test,xgb_rf)

In [ ]:
importance = pd.Series(xgb_rf.feature_importances_,index=X_train.columns).sort_values(ascending=False)
display(importance.head(20))
top_cols = list(importance.head(20).index)

## Support Vector Machines

In [ ]:
import os,glob,sys
folder ='../py_files/'
sys.path.append(os.path.abspath(folder))
import mod_5_functions as m5

In [ ]:
# X_train[top_cols]
X_train.shape

In [ ]:
# np.logspace(0.01, 10,0.1)
np.logspace(-1,1,3)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC,LinearSVC,NuSVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pipe = Pipeline([('sca',MinMaxScaler()),
                ('clf',SVC(probability=True))])


# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'sca__feature_range': [(0,1)],#,(-1,1)],
    'clf__C':[0.1,0.5,10],#list( np.logspace(-2,1,2)),
}

In [ ]:
timer=Timer()
timer.start()
search = RandomizedSearchCV(pipe, param_grid,n_iter=2)#, n_jobs=-1)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
timer.stop()

In [ ]:

svc = SVC(probability=True)

In [ ]:
timer = Timer()
timer.start()
svc.fit(X_train[top_cols],y_train)

print(svc.score(X_test[top_cols],y_test))
timer.stop()

In [ ]:
y_hat_test = svc.predict(X_test[top_cols])
evaluate_model(y_test,y_hat_test,X_test[top_cols], svc)#,conf_matrix_kws={'normalize':True})

# Pipelines

Scikit-learn has a class called [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that is very logical and versatile. We can break up the steps within a full process. But it'll help if we define what the different parts are.

## Estimator

This is any object in the pipeline that can can take in data and *estimate* (or **learn**) some parameters. 

This means regression and classification models are estimators but so are objects that transform the original dataset ([Transformers](pipeline_intro.ipynb#Transformer)) such as a standard scaling.

### Usage (Methods)

#### `fit`

All estimators estimate/learn by calling the `fit()` method by passing in the dataset. Other parameters can be passed in to "help" the estimator to learn. These are called **hyperparameters**, parameters used to tweak the learning process.

## Transformer

Some estimators can change the original data to something new, a **transformation**. You can think of examples of these **transformers** when you do scaling, data cleaning, or expanding/reducing on a dataset.

### Usage (Methods)

#### `transform`

Transformers will call the `transform()` method to apply the transformation to a dataset.

####  `fit_transform`

Remember that all estimators have a `fit()` method, so a transformer can use the `fit()` method to learn something about the given dataset. After learning with `fit()`, a transformation on the dataset can be made with the `transform()` method. 

An example of this would be a function that performs normalization on the dataset; the `fit()` method would learn the minimum and maximum of the dataset and the `transform()` method will scale the dataset.

When you call `fit` and `transform` with the same dataset, you can simply call the `fit_transform()` method. This essentially has the same results as calling `fit()` and then `transform()` on the dataset but possibly with some optimization and efficiencies baked in.

## Predictor

We've been using **predictors** whenever we've been making predictions with a classifier or regressor. We would use the `fit()` method to train our predictor object and then feed in new data to make predictions (based on what it learned in the fitting stage).

### Usage (Methods)

#### `predict`

As you probably can guess, the `predict()` method predicts results from a dataset given to it after being trained with a `fit()` method

#### `score`

Predictors also have a `score()` method that can be used to evaluate how well the predictor performed on a dataset (such as the test set).

# Example of Using a Pipeline

We can imagine doing the full steps planned out for a dataset. We _technically_ don't need to use the Pipeline class but it makes it much more manageable

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_digits, y_digits)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

In [ ]:
# Getting some data
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = iris.data
y = iris.target

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=27)

## Without the Pipeline class

In [ ]:
# Define transformers (will adjust/massage the data)
imputer = SimpleImputer(strategy="median") # replaces missing values
std_scaler = StandardScaler() # scales the data
pca = PCA()

# Define the classifier (predictor) to train
rf_clf = RandomForestClassifier()

# Have the classifer (and full pipeline) learn/train/fit from the data
X_train_filled = imputer.fit_transform(X_train)
X_train_scaled = std_scaler.fit_transform(X_train_filled)
X_train_reduce = pca.fit_transform(X_train_scaled)
rf_clf.fit(X_train_reduce, y_train)

# Predict using the trained classifier (still need to do the transformations)
X_test_filled = imputer.transform(X_test)
X_test_scaled = std_scaler.transform(X_test_filled)
X_test_reduce = pca.fit_transform(X_test_scaled)
y_pred = rf_clf.predict(X_test_reduce)

> Note that if we were to add more steps in this process, we'd have to change both the *training* and *testing* processes.

## With the Pipeline class

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), 
        ('std_scaler', StandardScaler()),
        ('pca', PCA()),
        ('rf_clf', RandomForestClassifier()),
])


# Train the pipeline (tranformations & predictor)
pipeline.fit(X_train, y_train)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted = pipeline.predict(X_test)

In [ ]:
model = pipeline.named_steps['rf_clf']

In [ ]:
m5.evaluate_model(y_test,predicted,model)

# Using SHAP and Shapely Values for Model Interpretation




- White Paper on Shapely Values:
    - https://arxiv.org/abs/1705.07874
    
- Blog Posts:
    - https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d

    - https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a


- Videos/Talks:
    - ["Open the Black Box: an intro to Model Interpretability with LIME and SHAP](https://youtu.be/C80SQe16Rao)
    

## Using SHAP

- Uses game theory to explain feature importance and how a feature steered a model's prediction(s) by removing each feature and seeing the effect on the error.

- SHAP has:
    - `TreeExplainer`:
        - compatible with sckit learn, xgboost, Catboost
    - `KernelExplainer`:
        - compatible with "any" model
        


- See [this blog post](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d) for intro to topic and how to use with trees

- For non-tree/random forest models [see this follow up post]( https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a)

        

### To Get Expanations for Trees:



- Import and initialize javascript:

```python
import shap 
shap.initjs()
```
1. Create a shap explainer using your fit model.

```python
explainer = shap.TreeExplainer(xgb_clf)
```

2. Get shapely values from explainer for your training data

```python
shap_values = explainer.shap_values(X_train,y_train)
```            

3. Select which type of the available plots you'd like to visualize

    
- **Types of Plots:**
    - `summary_plot()`
    - `dependence_plot()`
    - `force_plot()` for a given observation
    - `force_plot()` for all data

### Summary Plot

```python

## For normal bar graph of importance:
shap.summary_plot(shap_values,X_train,plot_type='bar')

## For detail Shapely value visuals:
shap.summary_plot(shap_values, X_train)
```

**`shap.summary_plot`**
> - Feature importance: Variables are ranked in descending order.
- Impact: The horizontal location shows whether the effect of that value is associated with a higher or lower prediction.
- Original value: Color shows whether that variable is high (in red) or low (in blue) for that observation.


**`shap.dependence_plot`**


```python
## To Auto-Select Feature Most correlated with a specific feature, just pass the desired feature's column name.

shap.dependence_plot('super_dist', shap_values, X_train)

## There is a way to specifically call out multiple features but I wasn't able to summarize it quickly for this nb
```

`shap.force_plot`

To show an individual data point's prediction and the factors pushing it towards one class or another

```python
## Just using np to randomly select a row

row = np.random.choice(range(len(X_train))
                       
shap.force_plot(explainer.expected_value, shap_values[row,:], X_train.iloc[row,:])
```

# Let's peak inside this black box
[SHAP Values](https://towardsdatascience.com/explain-your-model-with-the-shap-values-bc36aac4de3d)

[Explain Any Model with SHAP KernelExplaibner](https://towardsdatascience.com/explain-any-models-with-the-shap-values-use-the-kernelexplainer-79de9464897a)

In [ ]:
# !pip install -U shap
import shap
shap.initjs()

In [ ]:
X_shap = shap.sample(X_test,nsamples=5)
explainer = shap.KernelExplainer(svc.predict,X_shap)#.shap_values()
shap_values = explainer.shap_values(X_shap)
# # # rf_shap_values = shap.KernelExplainer(rf.predict,X_test)

In [ ]:
shap.summary_plot(shap_values,X_shap)

In [ ]:
shap.initjs()

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_shap)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_test)

# Using a Pipeline

Check out Aurélien Geron's notebook of an [end-to-end ml project](https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb) on his GitHub repo based around his book [_Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (2nd ed)_](https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/)